In [1]:
import pandas as pd
import numpy as np
import glob,os


dir = "D:\\WindData\\POD_Request_3541\\Project\\processed\\trimmed"

os.chdir(dir)

if not os.path.exists('final'):
    os.makedirs('final')
if not os.path.exists('final/data'):
        os.makedirs('final/data')


In [7]:
import pickle

In [2]:
labels = glob.glob("labels/*.csv")
dataSets = glob.glob("data/*.csv")
print(labels)



['labels\\0.csv', 'labels\\1.csv', 'labels\\10.csv', 'labels\\11.csv', 'labels\\12.csv', 'labels\\13.csv', 'labels\\14.csv', 'labels\\15.csv', 'labels\\16.csv', 'labels\\2.csv', 'labels\\3.csv', 'labels\\4.csv', 'labels\\5.csv', 'labels\\6.csv', 'labels\\7.csv', 'labels\\8.csv', 'labels\\9.csv']


In [3]:
def pullBatch(numBatches, batchSize, depth, labelFile, dataFile):
    dataList = []
    avgList = []
    skip = numBatches * batchSize * depth
    for i in range(batchSize):
        frameData = pd.read_csv(dataFile, header=None, skiprows=skip, nrows=depth)
        if len(frameData) < depth:
            return [], 0
        else:
            avgData = pd.read_csv(labelFile, header=None, skiprows=skip, nrows=depth)
            avg = avgData[0].mean()
            avgList.append(avg)
            dataList.append(frameData.values.tolist())
            skip += depth
    return dataList, avgList
    

In [4]:
batchSize = 32
depth = 60
labelList = []
batchCount = 0

for label, data in zip(labels, dataSets):
    numBatches = 0
    batch, avgList = pullBatch(numBatches, batchSize, depth, label, data)
    while not len(batch)==0:
        npArray = np.array(batch)
        labelList.append(avgList)
        np.save("final/data/" + str(batchCount) + ".npy", npArray)
        numBatches+=1
        batchCount+=1
        batch, avgList = pullBatch(numBatches, batchSize, depth, label, data)
    print("Round done")

Round done
Round done
Round done
Round done
Round done
Round done
Round done
Round done
Round done
Round done
Round done
Round done
Round done
Round done
Round done
Round done
Round done


In [ ]:
#frame = pd.DataFrame(labelList, )

In [5]:
print(labelList[0])

[604770.5, 604710.5, 604650.5, 604590.5, 604530.5, 604470.5, 604410.5, 604350.5, 604290.5, 604230.5, 604170.5, 604110.5, 604050.5, 603990.5, 603930.5, 603870.5, 603810.5, 603750.5, 603690.5, 603630.5, 603570.5, 603510.5, 603450.5, 603390.5, 603330.5, 603270.5, 603210.5, 603150.5, 603090.5, 603030.5, 602970.5, 602910.5]


In [6]:
print(len(labelList))

1716


In [9]:
pickle.dump(labelList, open("./final/labels.p", "wb"))